In [4]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 4.9 MB/s 


In [20]:
import os
import numpy 
import pandas 
import sys
import requests
import json
import time
import subprocess
import sys

In [5]:
import Bio
print(Bio.__version__)
from Bio import SeqIO
from Bio.SeqIO.QualityIO import FastqGeneralIterator

1.79


#Fetch Data

## Project: PRJNA277616
Secondary Study Accession: SRP055967

In [6]:
url = 'https://www.ebi.ac.uk/ena/portal/api/filereport'
dataset = 'PRJNA277616'

params = {'accession':dataset,
          'format':'json',
          'download':'false',
          'result':'read_run'}

r = requests.get(url,
                 params = params
                )

### Print API URL

In [7]:
r.url

'https://www.ebi.ac.uk/ena/portal/api/filereport?accession=PRJNA277616&format=json&download=false&result=read_run'

### API Data to DF

In [8]:
my_json = json.loads(r.text)
df = pandas.json_normalize(json.loads(r.text))
df = df.sort_values(by=['sra_bytes'], ascending=False)

In [9]:
df

,run_accession,fastq_ftp,fastq_bytes,fastq_md5,submitted_ftp,submitted_bytes,submitted_md5,sra_ftp,sra_bytes,sra_md5
1,SRR1909613,ftp.sra.ebi.ac.uk/vol1/fastq/SRR190/003/SRR190...,509176084;512857801,8010de0e8960a60c17c857ed72326986;b8d6b1ccb5e21...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR190/003/SRR1909613,735255968,ad799097cc631623377d50e15d5e86b8
6,SRR1926133,ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/003/SRR192...,443636019;448731045,5fe97ee4ee79b803ec351b2e49486e69;a6d022444fc9b...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR192/003/SRR1926133,698658517,8e7306f2eb4a5aabc37c3f1ee7773454
2,SRR1909637,ftp.sra.ebi.ac.uk/vol1/fastq/SRR190/007/SRR190...,476209436;482411937,76cd80cbe1bb130b71ff545944175a10;6600f4b4b8bff...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR190/007/SRR1909637,694463458,cf9ffb85ec3cca058f5f87315404e00d
8,SRR1926135,ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/005/SRR192...,453426565;455280077,064993e49f2505116d888d24badd1213;9541654cde667...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR192/005/SRR1926135,639102091,9e2e5f128fad8d74a34f6011bd044c5e
7,SRR1926134,ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/004/SRR192...,126633468;127279139,15994448aa63986990b296790fb07b6d;4610a7ef19498...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR192/004/SRR1926134,178889595,82e76da875e89ac808b298a21a9a82a6
0,SRR1867792,ftp.sra.ebi.ac.uk/vol1/fastq/SRR186/002/SRR186...,969299466;980437258,1a51cf21b35c6dbaa39c59458e095c11;27be060c3f312...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR186/002/SRR1867792,1392075613,1212cbb2094c871d08b182089ab41310
5,SRR1926132,ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/002/SRR192...,860705147;870451896,567a0485cf4fd45518d6d40a48ab4602;b3a8f51c7fceb...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR192/002/SRR1926132,1362585136,7c9c7fe9226093e913f21143e9f47160
9,SRR1926136,ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/006/SRR192...,880466468;881617202,5fff95da7170a33e4787a711e2034b81;d22d9775a5850...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR192/006/SRR1926136,1313869366,c17f66c0874cadacf5104bcbbc95862f
3,SRR1909638,ftp.sra.ebi.ac.uk/vol1/fastq/SRR190/008/SRR190...,818597025;829022213,1bf535cf7c087b91ee63c5ee8b179b93;d0cebabefcf7c...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR190/008/SRR1909638,1179953899,64357edc6c3749ba825b963357789284
4,SRR1909639,ftp.sra.ebi.ac.uk/vol1/fastq/SRR190/009/SRR190...,671504088;679436514,2fb29f0c80b7849c030ac01bf4b6f23e;409a5fbcf89c4...,,,,ftp.sra.ebi.ac.uk/vol1/srr/SRR190/009/SRR1909639,1040564099,79a78ab1825d55a2aa76ad6b9242f26a


### List Avaliable Runs

In [10]:
df['run_accession']

1    SRR1909613
6    SRR1926133
2    SRR1909637
8    SRR1926135
7    SRR1926134
0    SRR1867792
5    SRR1926132
9    SRR1926136
3    SRR1909638
4    SRR1909639
Name: run_accession, dtype: object

### Download Selected Runs

In [11]:
selected_runs = ['SRR1926134'] # This should be 10 to start but testing with 2
df[df['run_accession']==selected_runs[0]]
for i in range(len(selected_runs)):
  subset = df[df['run_accession']==selected_runs[i]]
  urls = subset['fastq_ftp'].values[0].split(";")
  for url in urls:
    print('Fetching: {}'.format(url))
    !wget {url}
    time.sleep(2)

Fetching: ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/004/SRR1926134/SRR1926134_1.fastq.gz
--2022-02-27 05:50:03--  http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/004/SRR1926134/SRR1926134_1.fastq.gz
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.197.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126633468 (121M) [application/octet-stream]
Saving to: ‘SRR1926134_1.fastq.gz’

SRR1926134_1.fastq. 100%[===================>] 120.77M   740KB/s    in 2m 55s  

2022-02-27 05:53:00 (705 KB/s) - ‘SRR1926134_1.fastq.gz’ saved [126633468/126633468]

Fetching: ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/004/SRR1926134/SRR1926134_2.fastq.gz
--2022-02-27 05:53:02--  http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR192/004/SRR1926134/SRR1926134_2.fastq.gz
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.138|:80... connected.
HTTP request sent, awa

### Unzip and Unpack

In [12]:
datapath = '/content/data/{}/'.format(dataset)
if not os.path.exists(datapath):
  os.makedirs(datapath)
for datafile in selected_runs:
  mygz= ''.join([datafile,'_1.fastq.gz'])
  myfastq= ''.join([datafile,'_1.fastq'])
  print('Unpacking: ' + mygz )
  if os.path.exists(mygz):
    !gzip -d {mygz} && mv {myfastq} {datapath}

  mygz= ''.join([datafile,'_2.fastq.gz'])
  myfastq= ''.join([datafile,'_2.fastq'])
  print('Unpacking: ' + mygz )
  if os.path.exists(mygz):
    !gzip -d {mygz} && mv {myfastq} {datapath}

Unpacking: SRR1926134_1.fastq.gz
Unpacking: SRR1926134_2.fastq.gz


# Download HISAT2

In [13]:
!wget https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download/hisat2-2.2.1-Linux_x86_64.zip

--2022-02-27 05:56:32--  https://cloud.biohpc.swmed.edu/index.php/s/oTtGWbWjaxsQ2Ho/download/hisat2-2.2.1-Linux_x86_64.zip
Resolving cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)... 129.112.9.92
Connecting to cloud.biohpc.swmed.edu (cloud.biohpc.swmed.edu)|129.112.9.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35050467 (33M) [application/zip]
Saving to: ‘hisat2-2.2.1-Linux_x86_64.zip’

hisat2-2.2.1-Linux_ 100%[===================>]  33.43M  43.5MB/s    in 0.8s    

2022-02-27 05:56:33 (43.5 MB/s) - ‘hisat2-2.2.1-Linux_x86_64.zip’ saved [35050467/35050467]



### Unzip Software

In [14]:
!unzip hisat2-2.2.1-Linux_x86_64.zip

Archive:  hisat2-2.2.1-Linux_x86_64.zip
   creating: hisat2-2.2.1/
  inflating: hisat2-2.2.1/hisat2-align-s-debug  
  inflating: hisat2-2.2.1/hisat2_extract_snps_haplotypes_VCF.py  
  inflating: hisat2-2.2.1/hisat2-build-l  
  inflating: hisat2-2.2.1/hisat2_extract_exons.py  
  inflating: hisat2-2.2.1/MANUAL.markdown  
   creating: hisat2-2.2.1/example/
   creating: hisat2-2.2.1/example/index/
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.7.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.6.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.1.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.3.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.5.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.2.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.8.ht2  
  inflating: hisat2-2.2.1/example/index/22_20-21M_snp.4.ht2  
   creating: hisat2-2.2.1/example/reference/
  inflating: hisat2-2.2.1/example/reference/22_20-21M.snp  
  

# Download Indexes

In [15]:
!wget https://genome-idx.s3.amazonaws.com/hisat/grch38_genome.tar.gz

--2022-02-27 05:57:21--  https://genome-idx.s3.amazonaws.com/hisat/grch38_genome.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 52.217.201.73
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|52.217.201.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4210306865 (3.9G) [binary/octet-stream]
Saving to: ‘grch38_genome.tar.gz’

grch38_genome.tar.g 100%[===================>]   3.92G  31.9MB/s    in 1m 58s  

2022-02-27 05:59:19 (34.2 MB/s) - ‘grch38_genome.tar.gz’ saved [4210306865/4210306865]



### Unpack Indexes

In [16]:
!tar -xvf grch38_genome.tar.gz

grch38/
grch38/genome.5.ht2
grch38/genome.2.ht2
grch38/make_grch38.sh
grch38/genome.3.ht2
grch38/genome.4.ht2
grch38/genome.7.ht2
grch38/genome.1.ht2
grch38/genome.6.ht2
grch38/genome.8.ht2


# Run Command as a Python Wrapper

In [17]:
read1 = "/content/data/PRJNA277616/SRR1926134_1.fastq"
read2 = "/content/data/PRJNA277616/SRR1926134_2.fastq"
outfile = "SRR1926134.sam"

In [19]:
cmd = ["/content/hisat2-2.2.1/hisat2",
       "-q",
       "-x",
       "$HISAT2_INDEXES/genome",
       "-1",
       read1,
       "-2",
      read2,
      "-S",
      outfile,
      "--time"]
print("Running Command: ")
print(*cmd, sep =" ")

Running Command: 
/content/hisat2-2.2.1/hisat2 -q -x $HISAT2_INDEXES/genome -1 /content/data/PRJNA277616/SRR1926134_1.fastq -2 /content/data/PRJNA277616/SRR1926134_2.fastq -S SRR1926134.sam --time


In [22]:
my_env = {**os.environ,
          'HISAT2_INDEXES': '/content/grch38/'}

process = subprocess.Popen(cmd, 
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.STDOUT,
                           env = my_env)
for line in process.stdout:
    sys.stdout.write(line)

Time loading forward index: 00:00:07
Time loading reference: 00:00:01
Multiseed full-index search: 00:06:32
2103171 reads; of these:
  2103171 (100.00%) were paired; of these:
    895462 (42.58%) aligned concordantly 0 times
    1155376 (54.93%) aligned concordantly exactly 1 time
    52333 (2.49%) aligned concordantly >1 times
    ----
    895462 pairs aligned concordantly 0 times; of these:
      164837 (18.41%) aligned discordantly 1 time
    ----
    730625 pairs aligned 0 times concordantly or discordantly; of these:
      1461250 mates make up the pairs; of these:
        1097505 (75.11%) aligned 0 times
        326778 (22.36%) aligned exactly 1 time
        36967 (2.53%) aligned >1 times
73.91% overall alignment rate
Time searching: 00:06:34
Overall time: 00:06:41


In [23]:
!ls

data	grch38_genome.tar.gz  hisat2-2.2.1-Linux_x86_64.zip  SRR1926134.sam
grch38	hisat2-2.2.1	      sample_data


# TODO

In [28]:
# Find some way to read .sam so it can be mapped
# DO NOT, try to open .sam files with cat or less.
# It's HUGE and  WILL cause a crash
# Maybe Samtools?

In [31]:
# Get size of output file
!du -shc {outfile}

1.4G	SRR1926134.sam
1.4G	total
